In [15]:
import numpy as np 
import cv2 as cv 
from sklearn.cluster import KMeans 
 
class Dictionary(object): 
    def __init__(self, name, img_filenames, num_words): 
        self.name = name #name of your dictionary 
        self.img_filenames = img_filenames #list of image filenames 
        self.num_words = num_words #the number of words 
         
        self.training_data = [] #training data used to learn clusters 
        self.words = [] #list of words, which are the centroids of clusters 
     
    def learn(self): 
        sift = cv.SIFT_create() 
         
        num_keypoints = [] #used to store the number of keypoints in each image 
         
        #load training images and compute SIFT descriptors 
        for filename in self.img_filenames: 
            img = cv.imread(filename) 
            img_gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY) 
            list_des = sift.detectAndCompute(img_gray, None)[1] 
            if list_des is None: 
                num_keypoints.append(0) 
            else: 
                num_keypoints.append(len(list_des)) 
                for des in list_des: 
                    self.training_data.append(des) 
             
        #cluster SIFT descriptors using K-means algorithm 
        kmeans = KMeans(self.num_words) 
        kmeans.fit(self.training_data) 
        self.words = kmeans.cluster_centers_ 
         
        #create word histograms for training images 
        training_word_histograms = [] #list of word histograms of all training images 
        index = 0 
        for i in range(0, len(self.img_filenames)): #for each file, create a histogram 
            histogram = np.zeros(self.num_words, np.float32) 
            #if some keypoints exist 
            if num_keypoints[i] > 0: 
                for j in range(0, num_keypoints[i]): 
                    histogram[kmeans.labels_[j + index]] += 1 
                index += num_keypoints[i] 
                histogram /= num_keypoints[i] 
                training_word_histograms.append(histogram) 
         
        return training_word_histograms
    
    def create_word_histograms(self, img_filenames): 
        sift = cv.SIFT_create() 
        histograms = [] 
         
        for filename in img_filenames: 
            img = cv.imread(filename) 
            img_gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY) 
            descriptors = sift.detectAndCompute(img_gray, None)[1] 
         
            histogram = np.zeros(self.num_words, np.float32) #word histogram  
         
            if descriptors is not None: 
                for des in descriptors: 
                    #find the best matching word 
                    min_distance = 1111111 #this can be any large number 
                    matching_word_ID = -1 #initialise ID with an impractical value 
                     
                    for i in range(0, self.num_words): #find the best matching word 
                        distance = np.linalg.norm(des - self.words[i]) 
                        if distance < min_distance: 
                            min_distance = distance 
                            matching_word_ID = i 
                     
                    histogram[matching_word_ID] += 1 
                 
                histogram /= len(descriptors) #make histogram a prob distribution 
         
            histograms.append(histogram) 
         
        return histograms 

In [16]:
import os 
 
foods = ['Cakes', 'Pasta', 'Pizza'] 
path = 'FoodImages/' 
training_file_names = [] 
training_food_labels = [] 
for i in range(0, len(foods)): 
    sub_path = path + 'Train/' + foods[i] + '/' 
    sub_file_names = [os.path.join(sub_path, f) for f in os.listdir(sub_path)] 
    sub_food_labels = [i] * len(sub_file_names) #create a list of N elements, all are i 
    training_file_names += sub_file_names 
    training_food_labels += sub_food_labels 
     
print(training_file_names) 
print(training_food_labels)

['FoodImages/Train/Cakes/cake1.png', 'FoodImages/Train/Cakes/cake10.jpg', 'FoodImages/Train/Cakes/cake11.jpg', 'FoodImages/Train/Cakes/cake12.jpg', 'FoodImages/Train/Cakes/cake13.jpg', 'FoodImages/Train/Cakes/cake14.jpg', 'FoodImages/Train/Cakes/cake15.jpg', 'FoodImages/Train/Cakes/cake16.jpg', 'FoodImages/Train/Cakes/cake17.jpg', 'FoodImages/Train/Cakes/cake18.jpg', 'FoodImages/Train/Cakes/cake19.jpg', 'FoodImages/Train/Cakes/cake2.png', 'FoodImages/Train/Cakes/cake20.jpg', 'FoodImages/Train/Cakes/cake21.jpg', 'FoodImages/Train/Cakes/cake22.jpg', 'FoodImages/Train/Cakes/cake23.jpg', 'FoodImages/Train/Cakes/cake24.jpg', 'FoodImages/Train/Cakes/cake25.jpg', 'FoodImages/Train/Cakes/cake26.jpg', 'FoodImages/Train/Cakes/cake27.jpg', 'FoodImages/Train/Cakes/cake28.jpg', 'FoodImages/Train/Cakes/cake29.jpg', 'FoodImages/Train/Cakes/cake3.png', 'FoodImages/Train/Cakes/cake30.jpg', 'FoodImages/Train/Cakes/cake4.jpg', 'FoodImages/Train/Cakes/cake5.jpg', 'FoodImages/Train/Cakes/cake6.jpg', 'FoodI

In [17]:
num_words = 50 
dictionary_name = 'food' 
dictionary = Dictionary(dictionary_name, training_file_names, num_words) 

In [18]:
training_word_histograms = dictionary.learn()

In [19]:
import pickle 
#save dictionary 
with open('food_dictionary.dic', 'wb') as f: #'wb' is for binary write 
    pickle.dump(dictionary, f)

In [20]:
with open('food_dictionary.dic', 'rb') as f: #'rb' is for binary read 
    dictionary = pickle.load(f) 

# KNN

In [21]:
from sklearn.neighbors import KNeighborsClassifier 
num_nearest_neighbours = 5 #number of neighbours 
knn = KNeighborsClassifier(n_neighbors = num_nearest_neighbours) 
knn.fit(training_word_histograms, training_food_labels) 

KNeighborsClassifier()

In [ ]:
test_file_names = [] 
test_food_labels = []

#load test images and create word histograms
for i in range(0, len(foods)): 
    sub_path = path + 'Test/' + foods[i] + '/' 
    sub_file_names = [os.path.join(sub_path, f) for f in os.listdir(sub_path)] 
    sub_food_labels = [i] * len(sub_file_names) #create a list of N elements, all are i 
    test_file_names += sub_file_names 
    test_food_labels += sub_food_labels 
    
word_histograms = dictionary.create_word_histograms(test_file_names) 
predicted_food_labels = knn.predict(word_histograms) 

Food label:  [0 0 0 0 0 0 0 0 1 0 0 1 2 0 0 2 0 0 2 0 0 2 0 0 2 2 0 2 0 1 1 2 2 2 1 1 1
 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 0 1 1 2 2 2 1 2 2 2 2 2
 2 2 2 2 2 1 2 2 2 2 2 2 2 2 1 1]


In [29]:
from sklearn.metrics import classification_report, confusion_matrix 
cm = confusion_matrix(test_food_labels, predicted_food_labels) 
print(cm) 
print(classification_report(test_food_labels, predicted_food_labels))

[[20  3  7]
 [ 0 26  4]
 [ 1  7 22]]
              precision    recall  f1-score   support

           0       0.95      0.67      0.78        30
           1       0.72      0.87      0.79        30
           2       0.67      0.73      0.70        30

    accuracy                           0.76        90
   macro avg       0.78      0.76      0.76        90
weighted avg       0.78      0.76      0.76        90



In [ ]:
num_nearest_neighbours = 10 #number of neighbours 
knn = KNeighborsClassifier(n_neighbors = num_nearest_neighbours) 
knn.fit(training_word_histograms, training_food_labels) 
predicted_food_labels = knn.predict(word_histograms) 
cm = confusion_matrix(test_food_labels, predicted_food_labels) 
print(cm) 
print(classification_report(test_food_labels, predicted_food_labels))

[[17  3 10]
 [ 0 26  4]
 [ 0  7 23]]
              precision    recall  f1-score   support

           0       1.00      0.57      0.72        30
           1       0.72      0.87      0.79        30
           2       0.62      0.77      0.69        30

    accuracy                           0.73        90
   macro avg       0.78      0.73      0.73        90
weighted avg       0.78      0.73      0.73        90



In [38]:
num_nearest_neighbours = 15 #number of neighbours 
knn = KNeighborsClassifier(n_neighbors = num_nearest_neighbours) 
knn.fit(training_word_histograms, training_food_labels) 
predicted_food_labels = knn.predict(word_histograms) 
cm = confusion_matrix(test_food_labels, predicted_food_labels) 
print(cm) 
print(classification_report(test_food_labels, predicted_food_labels))

[[14  5 11]
 [ 0 27  3]
 [ 0  6 24]]
              precision    recall  f1-score   support

           0       1.00      0.47      0.64        30
           1       0.71      0.90      0.79        30
           2       0.63      0.80      0.71        30

    accuracy                           0.72        90
   macro avg       0.78      0.72      0.71        90
weighted avg       0.78      0.72      0.71        90



In [39]:
num_nearest_neighbours = 20 #number of neighbours 
knn = KNeighborsClassifier(n_neighbors = num_nearest_neighbours) 
knn.fit(training_word_histograms, training_food_labels) 
predicted_food_labels = knn.predict(word_histograms) 
cm = confusion_matrix(test_food_labels, predicted_food_labels) 
print(cm) 
print(classification_report(test_food_labels, predicted_food_labels))

[[14  7  9]
 [ 0 27  3]
 [ 0  7 23]]
              precision    recall  f1-score   support

           0       1.00      0.47      0.64        30
           1       0.66      0.90      0.76        30
           2       0.66      0.77      0.71        30

    accuracy                           0.71        90
   macro avg       0.77      0.71      0.70        90
weighted avg       0.77      0.71      0.70        90



In [40]:
num_nearest_neighbours = 25 #number of neighbours 
knn = KNeighborsClassifier(n_neighbors = num_nearest_neighbours) 
knn.fit(training_word_histograms, training_food_labels) 
predicted_food_labels = knn.predict(word_histograms) 
cm = confusion_matrix(test_food_labels, predicted_food_labels) 
print(cm) 
print(classification_report(test_food_labels, predicted_food_labels))

[[11 11  8]
 [ 0 26  4]
 [ 0  9 21]]
              precision    recall  f1-score   support

           0       1.00      0.37      0.54        30
           1       0.57      0.87      0.68        30
           2       0.64      0.70      0.67        30

    accuracy                           0.64        90
   macro avg       0.73      0.64      0.63        90
weighted avg       0.73      0.64      0.63        90



In [41]:
num_nearest_neighbours = 30 #number of neighbours 
knn = KNeighborsClassifier(n_neighbors = num_nearest_neighbours) 
knn.fit(training_word_histograms, training_food_labels) 
predicted_food_labels = knn.predict(word_histograms) 
cm = confusion_matrix(test_food_labels, predicted_food_labels) 
print(cm) 
print(classification_report(test_food_labels, predicted_food_labels))

[[12 11  7]
 [ 0 27  3]
 [ 0 10 20]]
              precision    recall  f1-score   support

           0       1.00      0.40      0.57        30
           1       0.56      0.90      0.69        30
           2       0.67      0.67      0.67        30

    accuracy                           0.66        90
   macro avg       0.74      0.66      0.64        90
weighted avg       0.74      0.66      0.64        90



# SVM

In [62]:
from sklearn import svm 
svm_classifier = svm.SVC(C = 10, #see slide 32 in week 4 handouts 
kernel = 'linear') #see slide 35 in week 4 handouts 
svm_classifier.fit(training_word_histograms, training_food_labels)
predicted_food_labels = svm_classifier.predict(word_histograms) 
cm = confusion_matrix(test_food_labels, predicted_food_labels) 
print(cm) 
print(classification_report(test_food_labels, predicted_food_labels))

[[23  2  5]
 [ 0 23  7]
 [ 1  6 23]]
              precision    recall  f1-score   support

           0       0.96      0.77      0.85        30
           1       0.74      0.77      0.75        30
           2       0.66      0.77      0.71        30

    accuracy                           0.77        90
   macro avg       0.79      0.77      0.77        90
weighted avg       0.79      0.77      0.77        90



In [63]:
svm_classifier = svm.SVC(C = 20, #see slide 32 in week 4 handouts 
kernel = 'linear') #see slide 35 in week 4 handouts 
svm_classifier.fit(training_word_histograms, training_food_labels)
predicted_food_labels = svm_classifier.predict(word_histograms) 
cm = confusion_matrix(test_food_labels, predicted_food_labels) 
print(cm) 
print(classification_report(test_food_labels, predicted_food_labels))

[[26  2  2]
 [ 0 23  7]
 [ 1  1 28]]
              precision    recall  f1-score   support

           0       0.96      0.87      0.91        30
           1       0.88      0.77      0.82        30
           2       0.76      0.93      0.84        30

    accuracy                           0.86        90
   macro avg       0.87      0.86      0.86        90
weighted avg       0.87      0.86      0.86        90



In [64]:
svm_classifier = svm.SVC(C = 30, #see slide 32 in week 4 handouts 
kernel = 'linear') #see slide 35 in week 4 handouts 
svm_classifier.fit(training_word_histograms, training_food_labels)
predicted_food_labels = svm_classifier.predict(word_histograms) 
cm = confusion_matrix(test_food_labels, predicted_food_labels) 
print(cm) 
print(classification_report(test_food_labels, predicted_food_labels))

[[26  2  2]
 [ 0 23  7]
 [ 1  4 25]]
              precision    recall  f1-score   support

           0       0.96      0.87      0.91        30
           1       0.79      0.77      0.78        30
           2       0.74      0.83      0.78        30

    accuracy                           0.82        90
   macro avg       0.83      0.82      0.82        90
weighted avg       0.83      0.82      0.82        90



In [65]:
svm_classifier = svm.SVC(C = 40, #see slide 32 in week 4 handouts 
kernel = 'linear') #see slide 35 in week 4 handouts 
svm_classifier.fit(training_word_histograms, training_food_labels)
predicted_food_labels = svm_classifier.predict(word_histograms) 
cm = confusion_matrix(test_food_labels, predicted_food_labels) 
print(cm) 
print(classification_report(test_food_labels, predicted_food_labels))

[[26  2  2]
 [ 0 24  6]
 [ 1  4 25]]
              precision    recall  f1-score   support

           0       0.96      0.87      0.91        30
           1       0.80      0.80      0.80        30
           2       0.76      0.83      0.79        30

    accuracy                           0.83        90
   macro avg       0.84      0.83      0.84        90
weighted avg       0.84      0.83      0.84        90



In [ ]:
svm_classifier = svm.SVC(C = 50, #see slide 32 in week 4 handouts 
kernel = 'linear') #see slide 35 in week 4 handouts 
svm_classifier.fit(training_word_histograms, training_food_labels)
predicted_food_labels = svm_classifier.predict(word_histograms) 
cm = confusion_matrix(test_food_labels, predicted_food_labels) 
print(cm) 
print(classification_report(test_food_labels, predicted_food_labels))

[[26  2  2]
 [ 0 24  6]
 [ 1  4 25]]
              precision    recall  f1-score   support

           0       0.96      0.87      0.91        30
           1       0.80      0.80      0.80        30
           2       0.76      0.83      0.79        30

    accuracy                           0.83        90
   macro avg       0.84      0.83      0.84        90
weighted avg       0.84      0.83      0.84        90



# AdaBoost

In [68]:
from sklearn.ensemble import AdaBoostClassifier 
adb_classifier = AdaBoostClassifier(n_estimators = 50, #number of weak classifiers 
random_state = 0) 
adb_classifier.fit(training_word_histograms, training_food_labels)
predicted_food_labels = adb_classifier.predict(word_histograms)
cm = confusion_matrix(test_food_labels, predicted_food_labels) 
print(cm) 
print(classification_report(test_food_labels, predicted_food_labels))

[[19  1 10]
 [ 0 20 10]
 [ 0  8 22]]
              precision    recall  f1-score   support

           0       1.00      0.63      0.78        30
           1       0.69      0.67      0.68        30
           2       0.52      0.73      0.61        30

    accuracy                           0.68        90
   macro avg       0.74      0.68      0.69        90
weighted avg       0.74      0.68      0.69        90



c:\Users\micha\anaconda3\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


In [69]:
adb_classifier = AdaBoostClassifier(n_estimators = 100, #number of weak classifiers 
random_state = 0) 
adb_classifier.fit(training_word_histograms, training_food_labels)
predicted_food_labels = adb_classifier.predict(word_histograms)
cm = confusion_matrix(test_food_labels, predicted_food_labels) 
print(cm) 
print(classification_report(test_food_labels, predicted_food_labels))

c:\Users\micha\anaconda3\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


[[20  1  9]
 [ 0 21  9]
 [ 1  2 27]]
              precision    recall  f1-score   support

           0       0.95      0.67      0.78        30
           1       0.88      0.70      0.78        30
           2       0.60      0.90      0.72        30

    accuracy                           0.76        90
   macro avg       0.81      0.76      0.76        90
weighted avg       0.81      0.76      0.76        90



In [70]:
adb_classifier = AdaBoostClassifier(n_estimators = 150, #number of weak classifiers 
random_state = 0) 
adb_classifier.fit(training_word_histograms, training_food_labels)
predicted_food_labels = adb_classifier.predict(word_histograms)
cm = confusion_matrix(test_food_labels, predicted_food_labels) 
print(cm) 
print(classification_report(test_food_labels, predicted_food_labels))

c:\Users\micha\anaconda3\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


[[21  1  8]
 [ 0 22  8]
 [ 1  1 28]]
              precision    recall  f1-score   support

           0       0.95      0.70      0.81        30
           1       0.92      0.73      0.81        30
           2       0.64      0.93      0.76        30

    accuracy                           0.79        90
   macro avg       0.84      0.79      0.79        90
weighted avg       0.84      0.79      0.79        90



In [71]:
adb_classifier = AdaBoostClassifier(n_estimators = 200, #number of weak classifiers 
random_state = 0) 
adb_classifier.fit(training_word_histograms, training_food_labels)
predicted_food_labels = adb_classifier.predict(word_histograms)
cm = confusion_matrix(test_food_labels, predicted_food_labels) 
print(cm) 
print(classification_report(test_food_labels, predicted_food_labels))

c:\Users\micha\anaconda3\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


[[21  3  6]
 [ 0 22  8]
 [ 0  3 27]]
              precision    recall  f1-score   support

           0       1.00      0.70      0.82        30
           1       0.79      0.73      0.76        30
           2       0.66      0.90      0.76        30

    accuracy                           0.78        90
   macro avg       0.81      0.78      0.78        90
weighted avg       0.81      0.78      0.78        90



In [72]:
adb_classifier = AdaBoostClassifier(n_estimators = 250, #number of weak classifiers 
random_state = 0) 
adb_classifier.fit(training_word_histograms, training_food_labels)
predicted_food_labels = adb_classifier.predict(word_histograms)
cm = confusion_matrix(test_food_labels, predicted_food_labels) 
print(cm) 
print(classification_report(test_food_labels, predicted_food_labels))

c:\Users\micha\anaconda3\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


[[20  2  8]
 [ 0 21  9]
 [ 1  3 26]]
              precision    recall  f1-score   support

           0       0.95      0.67      0.78        30
           1       0.81      0.70      0.75        30
           2       0.60      0.87      0.71        30

    accuracy                           0.74        90
   macro avg       0.79      0.74      0.75        90
weighted avg       0.79      0.74      0.75        90

